In [ ]:
# 1) ITNRODUCTION / BUSINESS PROBLEM

# I have just started my organic cosmetics company. I already have the prototypes for the cosmetics and a great website but am now looking for a good small commercial store in New York City from where I can start selling my physical products.

# Here the criterias I am looking for to find the best possible location for my organic cosmetics shop:

# - Space Size: between 300 and 700 square feet
# - Location should be near other commercial stores in NYC
# - Rent price: the least expensive as possible
# - Location should be near places that have been rated by people who like organic products

# To find the best possible place I will use Foursquare location data and people's reviews. 



# 2) DATA SECTION => Examples of Foursquare data I am planning to use:
# Venues in NYC that are for commercial use between 300 and 700 feet big
# Venues that are located / concentrated around other venues
# The Price of Venues with the lowest as possible rent if I can find it
# look at Venues that have been highly rated by users
# Users who like and have reviewed organic products in NYC to find out where these places and people are
